<a href="https://colab.research.google.com/github/usanovich/GI_Piragua/blob/main/Niveles_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cuaderno para extracción de datos de niveles a partir de la API del geoportal de Piragua-Corantioquia.**

In [ ]:
install.packages("urltools") # Instalación en sesión de librería para manejo de enlaces

Cargamos librerías necesarias:

In [ ]:
library(tidyverse)
library(urltools)
library(lubridate)
library(dplyr)
library(ggplot2)

Limpiamos el entorno global:

In [ ]:
rm(list = ls())

Definimos fechas entre las que se desea extraer los datos:

In [ ]:
comenzar = "2010-01-03"
finalizar = "2021-01-10"

Se lee el catálogo de estaciones de Piragua:

In [ ]:
estaciones = read.csv("http://www.piraguacorantioquia.com.co/redes_nuevo/images/Estaciones_Piragua.csv")
estaciones_limni$municipio = make.names(estaciones_limni[,4], unique = TRUE)

Se filtra por tipo (limnigráficas); se pueden aplicar filtros adicionales a esta línea: territorial o municipios, por ejemplo.

In [ ]:
estaciones_limni = estaciones %>%
  filter(tipo == "Limnígrafo")

Creamos la primera función de extracción de datos a partir de la API de Piragua.


In [ ]:
get_data_limni = function(estacion, inicio, final){
  url="https://www.piraguacorantioquia.com.co/api/nivel/1017?date_estacion__gte=2020-01-01&date_estacion__lt=2020-01-02&downloadfile"
  url2 = param_set(url, "date_estacion__gte", inicio)
  url3 = param_set(url2, "date_estacion__lt", final)
  data = read.csv(gsub("1017", estacion, url3))
  data = na.omit(data)
  data = data[!(data$fechas == ""), ] # Para borrar campos de fechas vacíos
  data$fechas = as.POSIXct(data$fechas, "%Y-%m-%d %H-%M")
  data$fechas = floor_date(data$fechas, "5 mins")
  data2 = data %>%
    group_by(fechas) %>%
    dplyr::summarise(nivel = mean(nivel)) #Cambiar muestra, nivel o caudal
}

Se itera por cada una de las estaciones requeridas y se aplican funciones adicionales de limpieza de datos.

In [ ]:
# Iteramos sobre las estaciones limnigráficas
  
limni = list() # Creamos una lista vacía
  
# Guardamos en lista de listas todos los datasets de las estaciones 
# para una territorial en particular
  
for (i in 1:nrow(estaciones_limni)) {
    
  estacion = estaciones_limni$codigo[i]
  limni[[i]] = get_data_limni(estacion, comenzar, finalizar)
    
  }
  
# Agregamos una columna de municipios a cada dataframe de la lista
  
for (i in 1:length(limni)) {
    
  if (nrow(limni[[i]]) > 0) {
    
      limni[[i]] = data.frame(limni[[i]], municipios = estaciones_limni[i,4])
  }
}



#Para eliminar listas con 0 filas
limni = limni[sapply(limni, nrow) > 0]


#Creamos un dataframe global con la lista de estaciones que tienen datos
limni = do.call(rbind, limni)

Se crean los resumenes (promedio, máximos y mínimos) para cada estación evaluada.

In [ ]:
#Agrupamos por municipios y hallamos su promedio
limni_prom = limni %>%
  group_by(municipios) %>%
  dplyr::summarise(medio = mean(nivel)) # Nivel, caudal o muestra

#Agrupamos por municipios y hallamos su máximo
limni_max = limni %>%
  group_by(municipios) %>%
  dplyr::summarise(máximo = max(nivel)) # Nivel, caudal o muestra

#Agrupamos por municipios y hallamos su mínimo
limni_min = limni %>%
  group_by(municipios) %>%
  dplyr::summarise(mínimo = min(nivel)) # Nivel, caudal o muestra

# Creamos dataset con resumen
limni_resumen = data.frame(limni_prom, limni_max$máximo,
                           limni_min$mínimo)
colnames(limni_resumen) = c("Municipio", "Promedio",
                            "Máximo", "Mínimo")

# Exportamos datos
write.table(limni_resumen,"limni.txt", sep = ";")